In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain==0.3.0 langchain-community==0.3.0 langchain-core==0.3.0 langchain-openai langchain-text-splitters faiss-cpu

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install fastapi uvicorn pyngrok nest_asyncio librosa

In [1]:
# 세션 다시 시작
import sys
sys.path.append('/content/drive/MyDrive/코딩/새싹해커톤')

In [2]:
import os, json
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from contextlib import asynccontextmanager
from ai_agent_v3 import MedicalAgent
from google.colab import userdata
from openai import OpenAI

api_key = userdata.get("OPEN_AI")
client = OpenAI(api_key=api_key)

NGROK_AUTH_TOKEN = userdata.get("NGROK_TOKEN")

self_report = {
    "한쪽 얼굴이 둔하고 손발이 저리거나 힘이 빠진다": 1,
    "한쪽 손에 힘이 없어 물건을 떨어뜨리거나 다리가 후들거려 비틀거린다": 1,
    "어떤 일이 언제 일어났는지 기억하지 못할 때가 있다": 2,
    "며칠 전에 들었던 이야기를 잊는다": 2,
    "손을 움직이거나 가만히 있을 때도 손이 떨린다": 1,
    "침대나 의자에서 일어날 때 몸이 무겁고 힘들다": 2,
    "옷 단추를 잠그거나 물건을 잡기 힘들다": 1,
    "근육 경련이 일어난다": 2
}

dir = "/content/drive/MyDrive/코딩/새싹해커톤"
model_save_path = '/content/drive/MyDrive/코딩/새싹해커톤/whisper_tiny_cls.pt'
audio_path = '/content/drive/MyDrive/코딩/새싹해커톤/talk_set2_collectorgs384_speakergs4917_speakergs4918_6_0_120.wav'
# rag_doc_path = '/content/drive/MyDrive/코딩/새싹해커톤/rag_practicce.txt'
rag_doc_path = '/content/drive/MyDrive/코딩/새싹해커톤/healthcare_result.json'
report_path = '/content/drive/MyDrive/코딩/새싹해커톤/dummydata1.json'
with open(report_path, 'r', encoding='utf-8') as f:
    report_history = json.load(f)

agent_instance = None

In [ ]:
# import json

# agent_instance = MedicalAgent(
#     api_key=api_key,
#     model_ckpt_path=model_save_path,
#     rag_doc_path=rag_doc_path
# )

# result = agent_instance.run(audio_path, self_report, report_history)

# print(json.dumps(result, ensure_ascii=False, indent=2))

# ## cpu로 2분 대화 결과 나오는 데 1분 걸림

In [ ]:
@asynccontextmanager
async def lifespan(app: FastAPI):
    global agent_instance
    print("🏥 Agent 초기화 중...")

    # 여기서 모델/파일 경로 틀리면 바로 에러 나고 서버 시작 안 됨
    agent_instance = MedicalAgent(
        api_key=api_key,
        model_ckpt_path=model_save_path,
        rag_doc_path=rag_doc_path
    )
    print("✅ 준비 완료")
    yield # 함수 일시 정지 후, 제어를 fastapi 서버에 넘기고, api 요청 받고 응답하는 동안 함수가 멈춰 있는 시점 표시

app = FastAPI(lifespan=lifespan)

class DiagnoseRequest(BaseModel):
    audio_path: str = Field(default=audio_path)
    self_report: dict = Field(default=self_report)
    report_history: dict | None = Field(default=None)

@app.post("/diagnose")
async def diagnose(request: DiagnoseRequest):
    # 에러 처리 없이 바로 실행 (문제 생기면 500 Error 리턴됨)
    # 여기가 지금 백엔드에서 보낸 audio 경로랑 리포트 받는 곳
    return agent_instance.run(request.audio_path, request.self_report, request.report_history)

@app.get("/")
def root():
    return {"status": "Running"}

# ---------------------------------------------------------
# 4. 서버 실행
# ---------------------------------------------------------
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()
public_url = ngrok.connect(8000)
print(f"\n🚀 [API URL] {public_url}/diagnose")

nest_asyncio.apply()

# [수정] uvicorn.run 대신 Config와 Server 객체를 만들어 await로 실행합니다.
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()

# https://controllably-copacetic-johan.ngrok-free.dev 여기로 들어가야